In [32]:
# Symbolic link from /usr/lib64/libnsl.so.2.0.0 to /usr/lib64/libnsl.so.1 required

from __future__ import print_function

#import os
#os.chdir("/usr/lib/oracle/19.3/client64/lib")

import traceback
import cx_Oracle
import numpy as np
import pandas as pd
import seaborn as sns
import time
import sys

def connectToOracleDB():
    global connection
    global cursor
    global username
    global password
    global host
    global service
    
    connection = 0
    cursor = 0
    
    host     = "dboramdms.spsi.co.za"
    username = "icemdmsudev"
    password = "V3ry$3Cr!t"
    service  = "mdms"
    
    try:
        
        connection = cx_Oracle.connect(username, password, host+"/"+service)
        
        cursor = connection.cursor()
        
        print("\nOracle DB connection initialized\n")
        
    except Exception as err:
        exc_type, exc_value, exc_tb = sys.exc_info()
        traceback.print_exception(exc_type, exc_value, exc_tb)
        exit()
           
def executeSQLquery( sql ):
    global cursor
    
    print("{}".format(sql))
    
    cursor.execute( sql )
    
    return cursor.fetchall()

def getMeterReadings( meterType, meterID, fromDt, toDt ):

    #meterID = "36a878da44b6434fbe12173931dceaf1"
    #fromDt = "2019-10-01 00:00:00" 
    #toDt = "2019-10-31 23:59:59"
    
    meterTypeCol = ""
    
    if( meterType == 'ELSTER'):
        meterTypeCol = "ELS_METER_ID"
    elif( meterType == "KAMSTRUP" ):
        meterTypeCol = "KAM_METER_ID"
    elif( meterType == "ECHELON" ):
        meterTypeCol = "NES_METER_ID"
    elif( meterType == "GENERIC" ):
        meterTypeCol = "GENERIC_METER_ID"
    
    results = executeSQLquery( """SELECT ENTRY_TIME, TOTAL_KWHP, TOTAL_KWHN, TOTAL_KVARP, TOTAL_KVARN FROM METER_READING
                                    WHERE {} = '{}'
                                    AND ENTRY_TIME BETWEEN TO_DATE('{}','YYYY-MM-DD HH24:MI:SS') AND TO_DATE('{}','YYYY-MM-DD HH24:MI:SS') 
                                    ORDER BY ENTRY_TIME ASC"""
                              .format( meterTypeCol, meterID, fromDt, toDt ) )
    
    #results = executeSQLquery( """SELECT * FROM ( SELECT METER_READING_ID, ENTRY_TIME, TOTAL_KWHP, TOTAL_KWHN, TOTAL_KVARP, TOTAL_KVARN FROM METER_READING
    #                               WHERE NES_METER_ID = '{}'
    #                              AND ( ENTRY_TIME BETWEEN TO_DATE('{}','YYYY-MM-DD HH24:MI:SS') AND
    #                                TO_DATE('{}','YYYY-MM-DD HH24:MI:SS')) ORDER BY ENTRY_TIME ASC )
    #                                WHERE ROWNUM <= {:d}"""
    #                          .format( meterID, fromDt, toDt, days*48 ) )
    
    return results

print("MDMS Functions Initialized")

MDMS Functions Initialized


In [33]:
connectToOracleDB()


Oracle DB connection initialized



In [34]:
mid = "4a6baa3d-5463-409e-9789-4a323dc0b6b4" # meter_sn = "30829119"

meterReadings = getMeterReadings( meterType="KAMSTRUP", meterID=mid, fromDt='2020-04-01 00:00:00', toDt='2020-04-15 23:59:59')
meterReadings = pd.DataFrame(data=meterReadings, columns=['ENTRY_TIME','TOTAL_KWHP','TOTAL_KWHN','TOTAL_KVARP','TOTAL_KVARN'])
meterReadings.head(10)

SELECT ENTRY_TIME, TOTAL_KWHP, TOTAL_KWHN, TOTAL_KVARP, TOTAL_KVARN FROM METER_READING
                                    WHERE KAM_METER_ID = '4a6baa3d-5463-409e-9789-4a323dc0b6b4'
                                    AND ENTRY_TIME BETWEEN TO_DATE('2020-04-01 00:00:00','YYYY-MM-DD HH24:MI:SS') AND TO_DATE('2020-04-15 23:59:59','YYYY-MM-DD HH24:MI:SS') 
                                    ORDER BY ENTRY_TIME ASC


,ENTRY_TIME,TOTAL_KWHP,TOTAL_KWHN,TOTAL_KVARP,TOTAL_KVARN
0,2020-04-01 00:00:00,38439.25,0.0,214.20,22.24
1,2020-04-01 00:30:00,38443.93,0.0,214.22,22.24
2,2020-04-01 01:00:00,38448.49,0.0,214.25,22.24
3,2020-04-01 01:30:00,38453.14,0.0,214.28,22.24
4,2020-04-01 02:00:00,38457.71,0.0,214.30,22.24
5,2020-04-01 02:30:00,38462.28,0.0,214.33,22.24
6,2020-04-01 03:00:00,38466.80,0.0,214.36,22.24
7,2020-04-01 03:30:00,38471.44,0.0,214.38,22.24
8,2020-04-01 04:00:00,38476.09,0.0,214.41,22.24
9,2020-04-01 04:30:00,38480.71,0.0,214.43,22.24


In [35]:
meterReadings.index = meterReadings['ENTRY_TIME']

In [36]:
meterReadings.head()

,ENTRY_TIME,TOTAL_KWHP,TOTAL_KWHN,TOTAL_KVARP,TOTAL_KVARN
ENTRY_TIME,,,,,
2020-04-01 00:00:00,2020-04-01 00:00:00,38439.25,0.0,214.20,22.24
2020-04-01 00:30:00,2020-04-01 00:30:00,38443.93,0.0,214.22,22.24
2020-04-01 01:00:00,2020-04-01 01:00:00,38448.49,0.0,214.25,22.24
2020-04-01 01:30:00,2020-04-01 01:30:00,38453.14,0.0,214.28,22.24
2020-04-01 02:00:00,2020-04-01 02:00:00,38457.71,0.0,214.30,22.24


## TOU Peak , Standard, & Off-Peak Register Consumption Calculations

In [148]:
touStartDate  = "2020-04-02"
touEndDate    = "2020-04-08"

reading_list_dates = pd.date_range( touStartDate, touEndDate, freq="1D" )

In [149]:
weekdayDates  = []
saturdayDates = []
sundayDates   = []

weekdayNames  = ["Monday","Tuesday","Wednesday","Thursday","Friday"]
saturdayNames = ["Saturday"]
sundayNames   = ["Sunday"]

for date in reading_list_dates:

    dayName = date.day_name(locale=None)
    
    if dayName in weekdayNames:
        weekdayDates.append( str(date) )
    elif dayName in saturdayNames:
        saturdayDates.append( str(date) )
    elif dayName in sundayNames:
        sundayDates.append( str(date) )

In [150]:
pk_reg_cons    = 0.0
std_reg_cons   = 0.0
offpk_reg_cons = 0.0

### Weekday TOU Schedule

In [151]:
weekdayTimeslots = []
weekdayTimeslots.append({"startTime":"00:00:00", "endTime":"06:00:00","type":"OP"})
weekdayTimeslots.append({"startTime":"06:00:00", "endTime":"07:00:00","type":"S"})
weekdayTimeslots.append({"startTime":"07:00:00", "endTime":"10:00:00","type":"P"})
weekdayTimeslots.append({"startTime":"10:00:00", "endTime":"18:00:00","type":"S"})
weekdayTimeslots.append({"startTime":"18:00:00", "endTime":"20:00:00","type":"P"})
weekdayTimeslots.append({"startTime":"20:00:00", "endTime":"22:00:00","type":"S"})

def weekday_sched( timeslot ):

    startTime = timeslot["startTime"]
    endTime   = timeslot["endTime"]
    touType   = timeslot["type"]

    week_sched = []
    for wk_date in weekdayDates:
        week_sched.append( {"touType":touType,"startDate":wk_date,"startTime":startTime,"endDate":wk_date,"endTime":endTime} )

    return week_sched

weekday_tou_schedules = []

for tslot in weekdayTimeslots:
    weekday_tou_schedules.append( weekday_sched( timeslot=tslot ) ) 

In [152]:
weekday_total_consumption = 0.0

for weekdaySched in weekday_tou_schedules:
    
    for sched in weekdaySched:
        
        startDateTime = sched["startDate"] + " " + sched["startTime"]
        startDT = pd.to_datetime(startDateTime)
        startDT = startDT - pd.Timedelta('2 hour')
        
        endDateTime   = sched["endDate"]   + " " + sched["endTime"]
        endDT = pd.to_datetime(endDateTime)
        endDT = endDT - pd.Timedelta('2 hour')

        readings_df = meterReadings[ startDT : endDT ]

        readingsDiff = readings_df['TOTAL_KWHP'].iloc[-1] - readings_df['TOTAL_KWHP'].iloc[0]
        
        weekday_total_consumption = weekday_total_consumption + readingsDiff
        
        touType = sched["touType"]
        
        if touType == "P":
            pk_reg_cons = pk_reg_cons + readingsDiff
        elif touType == "S":
            std_reg_cons = std_reg_cons + readingsDiff
        elif touType == "OP":
            offpk_reg_cons = offpk_reg_cons + readingsDiff

print("Weekday Total Consumption: {:.3f}".format(weekday_total_consumption) )

Weekday Total Consumption: 1021.970


### Saturday TOU Schedule

In [153]:
saturdayTimeslots = []
saturdayTimeslots.append({"startTime":"00:00:00", "endTime":"23:59:59", "type":"OP"})

def saturday_sched( timeslot ):

    startTime = timeslot["startTime"]
    endTime   = timeslot["endTime"]
    touType   = timeslot["type"]

    sat_sched = []
    for sat_date in saturdayDates:
        sat_sched.append( {"touType":touType,"startDate":sat_date,"startTime":startTime,"endDate":sat_date,"endTime":endTime} )
    
    return sat_sched

saturday_tou_schedules = []

for tslot in saturdayTimeslots:
    saturday_tou_schedules.append( saturday_sched( timeslot=tslot ) ) 

In [154]:
saturday_total_consumption = 0.0

for saturdaySched in saturday_tou_schedules:
    
    for sched in saturdaySched:

        startDateTime = sched["startDate"] + " " + sched["startTime"]
        startDT = pd.to_datetime(startDateTime)
        startDT = startDT - pd.Timedelta('2 hour')
        
        endDateTime   = sched["endDate"]   + " " + sched["endTime"]
        endDT = pd.to_datetime(endDateTime)
        endDT = endDT - pd.Timedelta('2 hour')

        readings_df = meterReadings[ startDT : endDT ]

        readingsDiff = readings_df['TOTAL_KWHP'].iloc[-1] - readings_df['TOTAL_KWHP'].iloc[0]

        saturday_total_consumption = saturday_total_consumption + readingsDiff
        
        touType = sched["touType"]
        
        if touType == "P":
            pk_reg_cons = pk_reg_cons + readingsDiff
        elif touType == "S":
            std_reg_cons = std_reg_cons + readingsDiff
        elif touType == "OP":
            offpk_reg_cons = offpk_reg_cons + readingsDiff
    
print("Saturday Total Consumption: {:.3f}".format(saturday_total_consumption) )

Saturday Total Consumption: 218.840


### Sunday TOU Schedule

In [155]:
sundayTimeslots = []
sundayTimeslots.append({"startTime":"00:00:00", "endTime":"23:59:59", "type":"OP"})

def sunday_sched( timeslot ):

    startTime = timeslot["startTime"]
    endTime   = timeslot["endTime"]
    touType   = timeslot["type"]

    sun_sched = []
    for sun_date in saturdayDates:
        sun_sched.append( {"touType":touType,"startDate":sun_date,"startTime":startTime,"endDate":sun_date,"endTime":endTime} )
    
    return sun_sched

sunday_tou_schedules = []

for tslot in sundayTimeslots:
    sunday_tou_schedules.append( sunday_sched( timeslot=tslot ) )

In [156]:
sunday_total_consumption = 0.0

for sundaySched in sunday_tou_schedules:
    
    for sched in sundaySched:

        startDateTime = sched["startDate"] + " " + sched["startTime"]
        startDT = pd.to_datetime(startDateTime)
        startDT = startDT - pd.Timedelta('2 hour')
        
        endDateTime   = sched["endDate"]   + " " + sched["endTime"]
        endDT = pd.to_datetime(endDateTime)
        endDT = endDT - pd.Timedelta('2 hour')

        readings_df = meterReadings[ startDT : endDT ]

        readingsDiff = readings_df['TOTAL_KWHP'].iloc[-1] - readings_df['TOTAL_KWHP'].iloc[0]

        sunday_total_consumption = sunday_total_consumption + readingsDiff
        
        touType = sched["touType"]
        
        if touType == "P":
            pk_reg_cons = pk_reg_cons + readingsDiff
        elif touType == "S":
            std_reg_cons = std_reg_cons + readingsDiff
        elif touType == "OP":
            offpk_reg_cons = offpk_reg_cons + readingsDiff
    
print("Sunday Total Consumption: {:.3f}".format(sunday_total_consumption) )

Sunday Total Consumption: 218.840


In [157]:
print("Peak Register Consumption: {:.3f}".format(pk_reg_cons))
print("Standard Register Consumption: {:.3f}".format(std_reg_cons))
print("Off-Peak Register Consumption: {:.3f}".format(offpk_reg_cons))

Peak Register Consumption: 231.200
Standard Register Consumption: 512.340
Off-Peak Register Consumption: 716.110


In [158]:
total_tou_consumption = pk_reg_cons + std_reg_cons + offpk_reg_cons
print("TOU Total Consumption: {:.3f}".format( total_tou_consumption ))

TOU Total Consumption: 1459.650


## Total kWh Consumption Calculation

In [160]:
touStartDateTime  =  touStartDate + " 00:00:00" 
touEndDateTime    =  touEndDate   + " 23:59:59"

readings_df = meterReadings[ touStartDateTime : touEndDateTime ]
totalkWhP_diff = readings_df['TOTAL_KWHP'].iloc[-1] - readings_df['TOTAL_KWHP'].iloc[0]
print("Total kWhP Register Consumption: {:.3f}".format(totalkWhP_diff))

Total kWhP Register Consumption: 1557.240
